In [1]:
import  numpy as np
import  matplotlib.pyplot as plt
import cv2
import pandas as pd
import requests
import cv2
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from sklearn.preprocessing import normalize

C:\Users\lenovo\AppData\Local\Temp\ipykernel_7428\4283929640.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
C:\Users\lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Define the pre-trained ResNet model
resnet = models.resnet50(pretrained=True)

# Remove the last layer
feature_extractor = torch.nn.Sequential(*(list(resnet.children())[:-1]))


C:\Users\lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
data=pd.read_csv('A2_Data.csv')
def preprocess_image_urls(image_column):
    image_column = image_column.apply(lambda x: x.replace("[", "").replace("]", "").replace("'", "").split(", "))
    return image_column

# Preprocess the 'Image' column
data['Image'] = preprocess_image_urls(data['Image'])
data.head()

,Unnamed: 0,Image,Review Text,Image_Vectors
0,3452,[https://images-na.ssl-images-amazon.com/image...,Loving these vintage springs on my vintage str...,"[array([[0.3948523 , 0.5592724 , 0.4393471 , ...."
1,1205,[https://images-na.ssl-images-amazon.com/image...,Works great as a guitar bench mat. Not rugged ...,"[array([[0.38690877, 0.5909363 , 0.47527072, ...."
2,1708,[https://images-na.ssl-images-amazon.com/image...,We use these for everything from our acoustic ...,"[array([[0.3948523 , 0.5592724 , 0.4393471 , ...."
3,2078,[https://images-na.ssl-images-amazon.com/image...,Great price and good quality. It didn't quite...,"[array([[0.38690877, 0.5909363 , 0.47527072, ...."
4,801,[https://images-na.ssl-images-amazon.com/image...,I bought this bass to split time as my primary...,"[array([[0.3948523 , 0.5592724 , 0.4393471 , ...."


In [5]:


def adjust_brightness_and_contrast(image, brightness=0, contrast=0):
        # Alpha controls contrast; Beta controls brightness.
        alpha = 1 + contrast / 127
        beta = brightness
        adjusted_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
        return adjusted_image

def preprocess(image_irl):
    response = requests.get(image_url)
    image = cv2.imdecode(np.frombuffer(response.content, np.uint8), -1)
    if image is None:
        return None
    image_height, image_width = image.shape[:2]


    #border removal
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _,thresh = cv2.threshold(gray,1,255,cv2.THRESH_BINARY)
    contours,hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    x,y,w,h = cv2.boundingRect(contours[0])
    image = image[y:y+h,x:x+w]
    image = cv2.resize(image, (80, 80))


    # Random flip
    flip_direction = np.random.choice(["horizontal", "vertical"])
    if flip_direction == "horizontal":
        image = cv2.flip(image, 1)  # 1: Flip horizontally
    elif flip_direction == "vertical":
        image = cv2.flip(image, 0)  # 0: Flip vertically

    

    #pixel normalisation
    def normalize_image(image):
        normalized_image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        return normalized_image

    image = normalize_image(image)
    

    #brightness and contrast adjustment
    def calculate_brightness_and_contrast(image):
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        mean, std_dev = cv2.meanStdDev(gray_image)
        return mean[0][0], std_dev[0][0]

    

    mean_brightness, std_dev_contrast = calculate_brightness_and_contrast(image)

    # Set a threshold for deciding whether to adjust brightness and contrast
    brightness_threshold = 20  
    contrast_threshold = 10 

    # Adjust brightness and contrast only if needed
    if mean_brightness < brightness_threshold or std_dev_contrast < contrast_threshold:
        brightness = 20  
        contrast = 20  
        adjusted_image = adjust_brightness_and_contrast(image, brightness=brightness, contrast=contrast)
    else:
        adjusted_image = image

    return image


image_url = data['Image'][200][0]
image = preprocess(image_url)


# plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
# plt.axis('off')
# plt.show()

In [6]:



def extract_features(image):
    img = cv2.resize(image, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_tensor = transforms.ToTensor()(img)
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    img_normalized = normalize(img_tensor)
    img_normalized = img_normalized.unsqueeze(0)
    with torch.no_grad():
        features = feature_extractor(img_normalized)
    features = features.view(features.size(0), -1)
    features = features.numpy()
    return features

extracted_features = extract_features(image)


normalized_features = normalize(extracted_features, axis=1, norm='l2')
print(len(normalized_features[0]))
print(normalized_features[0])


2048
[0.01791445 0.02736122 0.02200573 ... 0.01664585 0.02190027 0.01612165]


In [7]:
def imageTOvec(image_id):
    image_url = image_id
    # print(image_url)
    image = preprocess(image_url)
    if image is None:
        return None
    features = extract_features(image)
    return features

def cosine_similarity(image_id1, image_id2):
    features1 = imageTOvec(image_id1)
    features2 = imageTOvec(image_id2)
    similarity = np.dot(features1, features2.T)
    return similarity


['https://images-na.ssl-images-amazon.com/images/I/51OuPPtNnWL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/71mCH1uxgSL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/71pMtSbThnL._SY88.jpg']

https://images-na.ssl-images-amazon.com/images/I/71mCH1uxgSL._SY88.jpg


In [8]:
data['Image_Vectors'] = None
for i, row in data.iterrows():
    print(i)
    vectors = []
    for url in row['Image']:
        vector = imageTOvec(url)
        if vector is not None:
            vectors.append(vector)
    if len(vectors) ==0:
        # drop the whole row
        data = data.drop(i)
    else:
        data.at[i, 'Image_Vectors'] = vectors
    

# Print the updated DataFrame
print(data)

0
1
2


KeyboardInterrupt: 

In [8]:
data.head()

,Unnamed: 0,Image,Review Text,Image_Vectors
0,3452,[https://images-na.ssl-images-amazon.com/image...,Loving these vintage springs on my vintage str...,"[[[0.39485234, 0.55927205, 0.43934655, 0.38080..."
1,1205,[https://images-na.ssl-images-amazon.com/image...,Works great as a guitar bench mat. Not rugged ...,"[[[0.38690838, 0.59093606, 0.47527072, 0.42505..."
2,1708,[https://images-na.ssl-images-amazon.com/image...,We use these for everything from our acoustic ...,"[[[0.38690838, 0.59093606, 0.47527072, 0.42505..."
3,2078,[https://images-na.ssl-images-amazon.com/image...,Great price and good quality. It didn't quite...,"[[[0.38690838, 0.59093606, 0.47527072, 0.42505..."
4,801,[https://images-na.ssl-images-amazon.com/image...,I bought this bass to split time as my primary...,"[[[0.39485234, 0.55927205, 0.43934655, 0.38080..."


In [9]:
droped_data = data.dropna()

In [10]:
#  drop rows with review text being nan
print(droped_data)

     Unnamed: 0                                              Image  \
0          3452  [https://images-na.ssl-images-amazon.com/image...   
1          1205  [https://images-na.ssl-images-amazon.com/image...   
2          1708  [https://images-na.ssl-images-amazon.com/image...   
3          2078  [https://images-na.ssl-images-amazon.com/image...   
4           801  [https://images-na.ssl-images-amazon.com/image...   
..          ...                                                ...   
995        1265  [https://images-na.ssl-images-amazon.com/image...   
996        1882  [https://images-na.ssl-images-amazon.com/image...   
997        1547  [https://images-na.ssl-images-amazon.com/image...   
998        1004  [https://images-na.ssl-images-amazon.com/image...   
999        1306  [https://images-na.ssl-images-amazon.com/image...   

                                           Review Text  \
0    Loving these vintage springs on my vintage str...   
1    Works great as a guitar bench mat. Not

In [11]:
#  preprorcess revierw texts
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer as ps

# Download the stopwords corpus if you haven't already
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

def preprocess_review_text(review_text):
    # print(review_text)
    # print index of the reivew text
    review_text = str(review_text)
    review_text = review_text.lower()
    review_text = re.sub(r'[^\w\s]', '', review_text)
    # remove stop words
    review_text = ' '.join([word for word in review_text.split() if word not in stop_words])
    # do stemming
    review_text = ' '.join([ps().stem(word) for word in review_text.split()])
    return review_text



data['Review Text'] = data['Review Text'].apply(preprocess_review_text)

data.head()

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Unnamed: 0,Image,Review Text,Image_Vectors
0,3452,[https://images-na.ssl-images-amazon.com/image...,love vintag spring vintag strat good tension g...,"[[[0.39485234, 0.55927205, 0.43934655, 0.38080..."
1,1205,[https://images-na.ssl-images-amazon.com/image...,work great guitar bench mat rug enough abus ta...,"[[[0.38690838, 0.59093606, 0.47527072, 0.42505..."
2,1708,[https://images-na.ssl-images-amazon.com/image...,use everyth acoust bass ukulel know smaller mo...,"[[[0.38690838, 0.59093606, 0.47527072, 0.42505..."
3,2078,[https://images-na.ssl-images-amazon.com/image...,great price good qualiti didnt quit match radi...,"[[[0.38690838, 0.59093606, 0.47527072, 0.42505..."
4,801,[https://images-na.ssl-images-amazon.com/image...,bought bass split time primari bass dean edg m...,"[[[0.39485234, 0.55927205, 0.43934655, 0.38080..."


In [12]:
import numpy as np
from collections import Counter
import math

def tokenize(text):
    # Split text into tokens (words)
    return text.split()

def calculate_tf(text):
    # Calculate term frequency (TF) for each word in the text
    tokens = tokenize(text)
    word_count = Counter(tokens)
    total_words = len(tokens)
    tf = {word: count / total_words for word, count in word_count.items()}
    return tf

def calculate_idf(documents):
    # Calculate inverse document frequency (IDF) for each word
    total_documents = len(documents)
    all_words = set([word for document in documents for word in tokenize(document)])
    idf = {}
    for word in all_words:
        doc_count = sum([1 for document in documents if word in tokenize(document)])
        idf[word] = math.log10(total_documents / doc_count)
    return idf

def calculate_tfidf(text, idf):
    # Calculate TF-IDF for each word in the text using precomputed IDF values
    tf = calculate_tf(text)
    tfidf = {word: tf[word] * idf[word] for word in tf}
    return tfidf

# make tfidf form review text
dataset = data['Review Text'].tolist()

# Calculate IDF for the dataset
idf = calculate_idf(dataset)

# Calculate TF-IDF for each document
tfidf_matrix = []
for document in dataset:
    tfidf = calculate_tfidf(document, idf)
    tfidf_matrix.append(tfidf)



In [28]:
#  normalise the tfidf matrix

# tfidf_matrix = normalize(tfidf_matrix, axis=1, norm='l2')



#  print tfidk in a tablular manner
tfidf_df = pd.DataFrame(tfidf_matrix)
#  convert thge nan to zeros
tfidf_df = tfidf_df.fillna(0)
#  print the first 5 rows
print(tfidf_df.head())



       love   vintag   spring     strat      good   tension     great  \
0  0.060206  0.23263  0.28732  0.091102  0.045883  0.139794  0.032990   
1  0.000000  0.00000  0.00000  0.000000  0.029924  0.000000  0.021515   
2  0.000000  0.00000  0.00000  0.000000  0.000000  0.000000  0.000000   
3  0.000000  0.00000  0.00000  0.000000  0.057354  0.000000  0.041238   
4  0.000000  0.00000  0.00000  0.000000  0.000000  0.000000  0.010758   

     stabil     float     bridg  ...  mayer  importantli  toneprint  \
0  0.139794  0.168192  0.097901  ...    0.0          0.0        0.0   
1  0.000000  0.000000  0.000000  ...    0.0          0.0        0.0   
2  0.000000  0.000000  0.000000  ...    0.0          0.0        0.0   
3  0.000000  0.000000  0.000000  ...    0.0          0.0        0.0   
4  0.000000  0.000000  0.000000  ...    0.0          0.0        0.0   

   stringthru  stopflair  biggi  accord  screenshot  amazoncom  piti  
0         0.0        0.0    0.0     0.0         0.0        0.0 

In [18]:
#  calculate cosine similarity between two review texts

def cosine_similarity(text1, text2):

    tfidf1 = calculate_tfidf(text1, idf)
    tfidf2 = calculate_tfidf(text2, idf)

    # Calculate the dot product of tfidf1 and tfidf2
    dot_product = sum([tfidf1[word] * tfidf2[word] for word in tfidf1 if word in tfidf2])

    # Calculate the magnitude of tfidf1 and tfidf2
    magnitude1 = math.sqrt(sum([tfidf1[word]**2 for word in tfidf1]))
    magnitude2 = math.sqrt(sum([tfidf2[word]**2 for word in tfidf2]))

    # Calculate the cosine similarity
    similarity = dot_product / (magnitude1 * magnitude2)
    return similarity

#  calculate cosine similarity between two review texts
text1 = data['Review Text'][0]
text2 = data['Review Text'][1]
similarity = cosine_similarity(text1, text2)
print(similarity, "1")

0.011436966078170158 1


In [22]:
#  creaate a ranked index of the most similar review texts

def most_similar_reviews(query_text, num_results=5):

    # Calculate the cosine similarity between the query text and all the review texts
    similarities = [cosine_similarity(query_text, review_text) for review_text in dataset]

    # Sort the review texts by their similarity to the query text
    most_similar_indices = np.argsort(similarities)[::-1]

    # Return the top N most similar review texts
    return most_similar_indices[:num_results]

#  print the most similar review texts

query_text = data['Review Text'][0]
similar_reviews = most_similar_reviews(query_text)
print(similar_reviews)

#  print the texts of the most similar review texts
for i in similar_reviews:
    print(data['Review Text'][i])
    print('\n')


[  0 271 805 750 758]
love vintag spring vintag strat good tension great stabil float bridg want spring way go


nice solid spring defeinit silent easi instal black look cool pictur old uninstal spring next


fit great complaint tremolo spring cheap qualiti block attach bridg took put though wammi bar wont fit


great qualiti adjust tension well made


use fender lock tuner five year variou strat tele definit help tune stabil way faster restr break




In [23]:
print(cosine_similarity(data['Review Text'][0], data['Review Text'][271]))
print(cosine_similarity(data['Review Text'][0], data['Review Text'][805]))
print(cosine_similarity(data['Review Text'][0], data['Review Text'][750]))
print(cosine_similarity(data['Review Text'][0], data['Review Text'][758]))    

0.3270878334937966
0.23373038454347247
0.21895440353789145
0.151745632178107


In [24]:
print(data['Review Text'][0])

love vintag spring vintag strat good tension great stabil float bridg want spring way go


In [33]:
#  save the tfidf matrix to a csv file
tfidf_df.to_csv('tfidf.csv', index=False)

# load the tfidf matrix from the csv file
#loaded_tfidf_df = pd.read_csv('tfidf.csv')
#print(loaded_tfidf_df.head())

In [30]:
data.head()

,Unnamed: 0,Image,Review Text,Image_Vectors
0,3452,[https://images-na.ssl-images-amazon.com/image...,love vintag spring vintag strat good tension g...,"[[[0.39485234, 0.55927205, 0.43934655, 0.38080..."
1,1205,[https://images-na.ssl-images-amazon.com/image...,work great guitar bench mat rug enough abus ta...,"[[[0.38690838, 0.59093606, 0.47527072, 0.42505..."
2,1708,[https://images-na.ssl-images-amazon.com/image...,use everyth acoust bass ukulel know smaller mo...,"[[[0.38690838, 0.59093606, 0.47527072, 0.42505..."
3,2078,[https://images-na.ssl-images-amazon.com/image...,great price good qualiti didnt quit match radi...,"[[[0.38690838, 0.59093606, 0.47527072, 0.42505..."
4,801,[https://images-na.ssl-images-amazon.com/image...,bought bass split time primari bass dean edg m...,"[[[0.39485234, 0.55927205, 0.43934655, 0.38080..."


In [47]:
print(data['Image_Vectors'][0][0][0])

[0.39485234 0.55927205 0.43934655 ... 0.35669106 0.47609252 0.3402192 ]


In [10]:
import pickle

In [16]:
# with open('data.pkl', 'wb') as f:
#     pickle.dump(data, f)

# Load the dataset back
with open('data.pkl', 'rb') as f:
    loaded_data = pickle.load(f)



# read the csv file
loaded_tfidf_df = pd.read_csv('tfidf.csv')
# print(loaded_tfidf_df.head())
loaded_data.head()

,Unnamed: 0,Image,Review Text,Image_Vectors
0,3452,[https://images-na.ssl-images-amazon.com/image...,love vintag spring vintag strat good tension g...,"[[[0.39485234, 0.55927205, 0.43934655, 0.38080..."
1,1205,[https://images-na.ssl-images-amazon.com/image...,work great guitar bench mat rug enough abus ta...,"[[[0.38690838, 0.59093606, 0.47527072, 0.42505..."
2,1708,[https://images-na.ssl-images-amazon.com/image...,use everyth acoust bass ukulel know smaller mo...,"[[[0.38690838, 0.59093606, 0.47527072, 0.42505..."
3,2078,[https://images-na.ssl-images-amazon.com/image...,great price good qualiti didnt quit match radi...,"[[[0.38690838, 0.59093606, 0.47527072, 0.42505..."
4,801,[https://images-na.ssl-images-amazon.com/image...,bought bass split time primari bass dean edg m...,"[[[0.39485234, 0.55927205, 0.43934655, 0.38080..."


   Unnamed: 0                                              Image  \
0        3452  [https://images-na.ssl-images-amazon.com/image...   
1        1205  [https://images-na.ssl-images-amazon.com/image...   
2        1708  [https://images-na.ssl-images-amazon.com/image...   
3        2078  [https://images-na.ssl-images-amazon.com/image...   
4         801  [https://images-na.ssl-images-amazon.com/image...   

                                         Review Text  \
0  love vintag spring vintag strat good tension g...   
1  work great guitar bench mat rug enough abus ta...   
2  use everyth acoust bass ukulel know smaller mo...   
3  great price good qualiti didnt quit match radi...   
4  bought bass split time primari bass dean edg m...   

                                       Image_Vectors  
0  [[[0.39485234, 0.55927205, 0.43934655, 0.38080...  
1  [[[0.38690838, 0.59093606, 0.47527072, 0.42505...  
2  [[[0.38690838, 0.59093606, 0.47527072, 0.42505...  
3  [[[0.38690838, 0.59093606, 0.47

In [60]:
print(data['Image_Vectors'][0])

[array([[0.39485234, 0.55927205, 0.43934655, ..., 0.35669106, 0.47609252,
        0.3402192 ]], dtype=float32)]


In [64]:
print(data['Image'][1][2])

https://images-na.ssl-images-amazon.com/images/I/71domStNfIL._SY88.jpg
